In [ ]:
import numpy as np


# Funkcja do wyznaczania funkcji użyteczności dla wielu kryteriów (dyskretna wersja)
def utility_function(values, reference_points):
    """
    Przyjmuje wartości kryteriów dla alternatyw oraz punkty referencyjne.
    Ekstrapoluje funkcje użyteczności na cały zbiór alternatyw dla wielu kryteriów.
    
    :param values: Lista wartości alternatyw (np. [[x1_1, x1_2, ...], [x2_1, x2_2, ...], ...])
    :param reference_points: Lista punktów referencyjnych dla każdego kryterium
    :return: Funkcja użyteczności dla każdej alternatywy i kryterium
    """
    # Dla każdego kryterium obliczamy funkcję użyteczności (normalizacja)
    utility_values = []
    for i in range(len(reference_points[0])):  # Iterujemy po kryteriach
        ref_values = [ref[i] for ref in reference_points]  # Zbiór punktów referencyjnych dla kryterium i
        min_ref = np.min(ref_values)
        max_ref = np.max(ref_values)
        utility_values.append([(value[i] - min_ref) / (max_ref - min_ref) for value in values])
    return np.array(utility_values).T  # Zwracamy wartości użyteczności dla wszystkich alternatyw

# Funkcja do porównania alternatyw (dyskretna wersja)
def ustar_dyskretne(values, reference_points, lambda_param=1):
    """
    Metoda UTA Star dla wersji dyskretnej (wielokryterialnej).
    
    :param values: Lista wartości alternatyw w przestrzeni kryteriów
    :param reference_points: Lista punktów referencyjnych dla każdego kryterium
    :param lambda_param: Współczynnik λ
    :return: Wybrana alternatywa
    """
    # Wyznaczanie funkcji użyteczności
    utility_values = utility_function(values, reference_points)
    
    # Wybór rozwiązania kompromisowego - wybieramy alternatywę z maksymalną sumą użyteczności
    summed_utility = np.sum(utility_values, axis=1)
    best_solution = np.argmax(summed_utility)  # Wybieramy alternatywę z największą sumą funkcji użyteczności
    return best_solution, utility_values

# Przykład użycia (wersja dyskretna)
values = np.array([[10, 20], [15, 25], [20, 30], [25, 35], [30, 40]])  # Wartości alternatyw w przestrzeni 2 kryteriów
reference_points = np.array([[5, 10], [20, 30], [35, 40]])  # Punkty referencyjne w przestrzeni 2 kryteriów
best_solution, utility_values = ustar_dyskretne(values, reference_points)
print(f"Najlepsza alternatywa (dyskretna): {values[best_solution]} z funkcją użyteczności {utility_values[best_solution]}")


In [ ]:
from scipy.interpolate import interp1d

# Funkcja do wyznaczania funkcji użyteczności dla wielu kryteriów (ciągła wersja)
def utility_function_continuous(values, reference_points, use_polynomial=False):
    """
    Aproksymuje funkcje użyteczności w przestrzeni ciągłej dla wielu kryteriów.
    
    :param values: Lista wartości kryteriów dla alternatyw
    :param reference_points: Lista punktów referencyjnych
    :param use_polynomial: Jeśli True, używa aproksymacji wielomianowej, w przeciwnym razie interpolacja liniowa
    :return: Funkcje użyteczności
    """
    # Dla każdego kryterium stosujemy interpolację lub aproksymację
    utility_values = []
    for i in range(len(reference_points[0])):  # Iterujemy po kryteriach
        ref_values = [ref[i] for ref in reference_points]  # Zbiór punktów referencyjnych dla kryterium i
        if use_polynomial:
            # Przy użyciu funkcji aproksymacji wielomianowej
            coef = np.polyfit(ref_values, [0, 1, 0], 2)  # Przykład: funkcja kwadratowa
            poly_func = np.poly1d(coef)
            utility_values.append(poly_func(values[:, i]))
        else:
            # Interpolacja liniowa
            interp_func = interp1d(ref_values, [0, 1, 0], kind='linear', fill_value="extrapolate")
            utility_values.append(interp_func(values[:, i]))
    return np.array(utility_values).T  # Zwracamy funkcje użyteczności dla wszystkich alternatyw

# Funkcja do porównania alternatyw (ciągła wersja)
def ustar_ciagle(values, reference_points, lambda_param=1, use_polynomial=False):
    """
    Metoda UTA Star dla wersji ciągłej (wielokryterialnej).
    
    :param values: Lista wartości alternatyw w przestrzeni kryteriów
    :param reference_points: Lista punktów referencyjnych dla każdego kryterium
    :param lambda_param: Współczynnik λ
    :param use_polynomial: Jeśli True, stosuje aproksymację wielomianową, w przeciwnym razie interpolację
    :return: Wybrana alternatywa
    """
    # Wyznaczanie funkcji użyteczności
    utility_values = utility_function_continuous(values, reference_points, use_polynomial)
    
    # Wybór rozwiązania kompromisowego - wybieramy alternatywę z maksymalną sumą użyteczności
    summed_utility = np.sum(utility_values, axis=1)
    best_solution = np.argmax(summed_utility)  # Wybieramy alternatywę z największą sumą funkcji użyteczności
    return best_solution, utility_values

# Przykład użycia (wersja ciągła)
values_continuous = np.random.uniform(0, 100, (10, 2))  # Przykładowe wartości alternatyw w przestrzeni 2 kryteriów
reference_points_continuous = np.array([[20, 30], [50, 60], [80, 90]])  # Punkty referencyjne w przestrzeni 2 kryteriów
best_solution_continuous, utility_values_continuous = ustar_ciagle(values_continuous, reference_points_continuous, use_polynomial=True)
print(f"Najlepsza alternatywa (ciągła): {values_continuous[best_solution_continuous]} z funkcją użyteczności {utility_values_continuous[best_solution_continuous]}")
